# Scrape Data

In [ ]:
!pip install scrapy
!scrapy startproject myproject
%cd /content/myproject/myproject


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.7 MB/s eta 0:00:00
New Scrapy project 'myproject', using template directory '/usr/local/lib/python3.11/dist-packages/scrapy/templates/project', created in:
    /content/myproject

You can start your first spider with:
    cd myproject
    scrapy genspider example example.com
/content/myproject/myproject


In [ ]:
spider_code = """import scrapy
import time

class WritingExamUrlSpider(scrapy.Spider):
    name = "writing-exam-url"
    base_url = "https://writing9.com"
    start_urls = []

    for i in [4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9]:
        for j in range(0, 1001):
          start_urls.append(f"{base_url}/band/{i}/{j}")

    def parse(self, response):
        list_data = response.xpath('//div[@class="jsx-1611283254 list"]')

        #check list contains item
        is_items = list_data.xpath('.//div[@class="jsx-2134295183 question-wrap"]') is not None

        if is_items:
            band = list_data.xpath('.//div[@class="jsx-3243968521 band"]')[0].xpath('.//text()').get().strip()  # First element
            rows = list_data.xpath('.//div[@class="jsx-2134295183 question-wrap"]')
            data = []
            for row in rows:
                href_url =  row.xpath('.//a[@href]/@href').get()
                data.append(href_url)
            yield {
              'band':band,
              'url':data
            }
"""

with open("/content/myproject/myproject/spiders/writing_exam_url_spider.py", "w") as f:
    f.write(spider_code)

print("Spider file created successfully!")

!scrapy crawl writing-exam-url -o url-data.json -s LOG_LEVEL=ERROR


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  File "/usr/local/lib/python3.11/dist-packages/scrapy/utils/python.py", line 368, in __next__
    return next(self.data)
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/scrapy/utils/python.py", line 368, in __next__
    return next(self.data)
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/scrapy/core/spidermw.py", line 106, in process_sync
    yield from iterable
  File "/usr/local/lib/python3.11/dist-packages/scrapy/spidermiddlewares/referer.py", line 379, in <genexpr>
    return (self._set_referer(r, response) for r in result)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/scrapy/core/spidermw.py", line 106, in process_sync
    yield from iterable
  File "/usr/local/lib/python3.11/dist-packages/scrapy/spidermiddlewares/urllength.py", line 57, in <genexpr>
    return (r for r in result if self._filter(r, spider))


# Scrape Data from URLs



In [ ]:
spider_code = r"""import scrapy
from scrapy.selector import Selector
import json
import time

class WritingExamSpider(scrapy.Spider):
    name = "writing-exam"

    base_url = "https://writing9.com"

    with open('url-data.json', 'r', encoding='utf-8') as file:
      data = json.load(file)  # Load JSON into a Python dictionary
    start_urls = []
    for entry in data:
      for url in entry['url']:
        start_urls.append(base_url + url)

    def parse(self, response):
        header = response.xpath('//h1[@class="jsx-3594441866 h4 "]/text()').get()

        analyzer  = response.xpath('//aside[@class="jsx-352101131 page-text__analyzer"]')

        linking_word = analyzer.xpath('.//div[@class="jsx-597499421 highlight-legends__item highlight-legends__item_linkingwords"]')

        linking_word_counter = linking_word.xpath('.//div[@class="jsx-597499421 highlight-legends__item-counter"]/text()').get()

        word_repetition = analyzer.xpath('.//div[@class="jsx-597499421 highlight-legends__item highlight-legends__item_repeatedwords"]')

        word_repetition_counter = word_repetition.xpath('.//div[@class="jsx-597499421 highlight-legends__item-counter"]/text()').get()

        grammar_mistakes = analyzer.xpath('.//div[@class="jsx-597499421 highlight-legends__item highlight-legends__item_mistake"]')

        grammar_mistakes_counter = grammar_mistakes.xpath('.//div[@class="jsx-597499421 highlight-legends__item-counter"]/text()').get()

        analyzer_containters = analyzer.xpath('.//div[@class="jsx-196542766 page-draft-text-analyzer__section-container"]')

        analyzer_secions = []
        index_key_map = {1: 'cc', 2: 'lr', 3: 'gr', 4: 'ta'}

        def extract_data(container, key):
          title = ''.join(container.xpath('.//span[@class="jsx-2601907021 caps"]/text()').getall()).strip()
          if key == 'cc':
            title = title.replace("Coherence and Cohesion: ","")
          elif key == 'lr':
            title = title.replace("Lexical resource: ","")
          elif key == 'gr':
            title = title.replace("Grammatical Range: ","")
          elif key == 'ta':
            title = title.replace("Task Achievement: ","")
          rows = container.xpath('.//div[@class="jsx-196542766 page-draft-text-analyzer__row"]')

          aspects = []

          if key == 'ta':  # Skip last row for 'ta'
            rows = rows[:-1]

          for row in rows:
              score = row.xpath('.//div[@class="jsx-1228298304 circle circle_color_red" or @class="jsx-1228298304 circle circle_color_green"]/text()').get()
              description = row.xpath('text()').get().strip()
              aspects.append({description: score})

          analyzer_secions.append({key: float(title), 'aspects': aspects})

        for index, container in enumerate(analyzer_containters[1:], start=1):
            if index in index_key_map:
                extract_data(container, index_key_map[index])
        def calculate_ielts_writing_score(cc,lr,gr,ta):
          average_score = sum([cc, lr, gr, ta]) / 4
          final_score = round(average_score * 2) / 2
          return final_score

        final_score = calculate_ielts_writing_score(
            cc=analyzer_secions[0].get('cc'),
            lr=analyzer_secions[1].get('lr'),
            gr=analyzer_secions[2].get('gr'),
            ta=analyzer_secions[3].get('ta')
        )

        contexts = response.xpath('//div[@class="jsx-1655112880 content-editable undefined"]/node()')

        in_nodes = []
        for node in contexts:
          if not hasattr(node.root, "tag"):
            text = node.get()
          else:
            text = node.xpath('.//span[@class="jsx-1153305423 text"]/text()').get()
          if text is not None:
            in_nodes.append(text)

        input_text = "".join(in_nodes)
        input_texts =  input_text.split("\r\n\r\n")

        out_nodes = []
        for node in contexts:
          if not hasattr(node.root, "tag"):
            text = node.get()
          else:
            type = ""
            class_attr = node.xpath('.//@class').get()  # Get class attribute

            if class_attr:
              if "error" in class_attr:
                type = "error"
              elif "repeated-word" in class_attr:
                type = "repeated-word"
              elif "linking-words" in class_attr:
                type = "linking-words"

            text = ""
            incorrect_text = node.xpath('.//span[@class="jsx-1153305423 text"]/text()').get()
            type_error = node.xpath('.//div[@class="jsx-1153305423 title"]/text()').get()
            strings = node.xpath('.//div[@class="jsx-1153305423 suggestion"]//text()').getall()
            suggestion = ", ".join(strings)
            ignore_case = ["correct article usage", "verb problem", "change preposition", "remove the comma",
                           "change the article", "change the verb form", "correct pronoun usage", "fix the agreement mistake",
                           "correct word choice", "remove the article", "remove the preposition", "remove the redundancy",
                           "correct the article-noun agreement"]
            if incorrect_text is not None and type_error is not None:
              if suggestion != "":
                if (type_error.lower() in ignore_case and "apply" in suggestion) or "unnecessary verb" in type_error.lower():
                  text += " [ "+ type + ": {" + incorrect_text + "} - " + type_error + "]"
                else:
                  text += " [ "+ type + ": {" + incorrect_text + "} - " + type_error + " - " + suggestion + "]"
              else:
                text += " [ "+ type + ": {" + incorrect_text + "} - " + type_error + "]"
          if text is not None:
            out_nodes.append(text)

        output_text = "".join(out_nodes)
        output_texts =  output_text.split("\r\n\r\n")

        yield {
          'question':header,
          'linking_word_counter':linking_word_counter,
          'word_repetition_counter':word_repetition_counter,
          'grammar_mistakes_counter':grammar_mistakes_counter,
          'analyzer_secions':analyzer_secions,
          'input_text':input_texts,
          'output_text':output_texts
        }
"""

with open("/content/myproject/myproject/spiders/writing_exam_spider.py", "w") as f:
    f.write(spider_code)

print("Spider file created successfully!")

!scrapy crawl writing-exam -o exam-data.json -s LOG_LEVEL=ERROR


Spider file created successfully!
2025-03-13 11:24:51 [scrapy.core.scraper] ERROR: Spider error processing <GET https://writing9.com/text/66825f9f8077b700112b5ac9-in-the-future-nobody-will-buy-printed-newspapers-or-books-because-they-will-be-able-to-read-everythi> (referer: None)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/scrapy/utils/defer.py", line 327, in iter_errback
    yield next(it)
          ^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/scrapy/utils/python.py", line 368, in __next__
    return next(self.data)
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/scrapy/utils/python.py", line 368, in __next__
    return next(self.data)
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/scrapy/core/spidermw.py", line 106, in process_sync
    yield from iterable
  File "/usr/local/lib/python3.11/dist-packages/scrapy/spidermiddlewares/referer.py", line 379, in <genexpr>
    return (self.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import json

# Load JSON file
json_file = "/content/drive/MyDrive/Colab Notebooks/exam-data.json"  # Replace with your actual file path
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert JSON data to Pandas DataFrame
df = pd.DataFrame(data)


In [8]:
df["input"] = "###Question: " + df["question"] + "###Answer: " + df["input_text"].apply(lambda x: " ".join(x)) + " ###"
# + ".lwc:" + df["linking_word_counter"] +  " .wrc:"  + df["word_repetition_counter"] + " .gmc" + df["grammar_mistakes_counter"]
import re
def replace_criteria(text):
    replacements = {
        "Logical structure": "LS",
        "Introduction & conclusion present": "ICP",
        "Supported main points": "SMP",
        "Accurate linking words": "ALW",
        "Variety in linking words": "VILW",
        "Varied vocabulary": "VV",
        "Accurate spelling & word formation": "ASWF",
        "Mix of complex & simple sentences": "MCSS",
        "Clear and correct grammar": "CCG",
        "Complete response": "CR",
        "Clear & comprehensive ideas": "CCI",
        "Relevant & specific examples": "RSE"
    }

    for key, value in replacements.items():
        text = re.sub(rf'\b{re.escape(key)}\b', value, text)

    return text
def extract_aspects(analyzer_sections):
    result = []
    for section in analyzer_sections:
        section_name = list(section.keys())[0]
        score = section[section_name]
        aspects = " ".join([f"{list(a.keys())[0]}: {float(list(a.values())[0])}" for a in section["aspects"]])
        result.append(f"{section_name.upper()}: {score} {aspects}")
    result = " ".join(result)
    modified_result = replace_criteria(result)
    return modified_result

df["output"]= "###Analysis: " + df["analyzer_secions"].apply(extract_aspects) + " ###"


In [9]:
pd.set_option("display.max_colwidth", None)
df["output"].head(1)

,output
0,###Analysis: CC: 4.0 LS: 4.0 ICP: 5.0 SMP: 4.0 ALW: 9.0 VILW: 9.0 LR: 4.0 VV: 7.0 ASWF: 4.0 GR: 4.0 MCSS: 9.0 CCG: 4.0 TA: 4.0 CR: 5.0 CCI: 5.0 RSE: 4.0 ###


In [10]:
df["input"].head(1)

,input
0,"###Question: It is neither possible nor useful to provide university places for a high proportion of young people. To what extent do you agree or disagree?###Answer: These days, the government can't provide university places for a high precentage of young people. I a gree of this point, so in this essay, I am going to discuss this veiw and give an example. In the one hand, that will effect of economic, each country has a plan to improve all services that young people need, such as education in college and at school. While, it is a hard to provide site for them at university, they should study well at school, and they must to get a high score, that support them to go to high education, as college and university. Also, the most country need private market to enter in university, for example in Oman , if students need to go to Suhar university, they should get a high precentage as 98%. On the other hand, nowadays, students study hard, because they want to study in high education, and the government support them by giving them money, and provide some services for them. For instance, hostel, and bus. Moreover, it is very difficult that the govrnment provide for them place in the college or university, due to teachers and students need more services , as well as they want salaries and a high conidition. In addtion, the economic will decline, if they provide free place for them, they will suffer in the future , they can't get on job. In conclusion, I believe that if anyone need to study in a high education, they must study well and they will get care and support from the government. Thus, they will achive their goal without feel uncomfortable. ###"


In [11]:
df[["input", "output"]].to_csv("processed_data.csv", index=False, encoding="utf-8")